In [1]:
from typing import List
from collections import deque
import bisect


class Router:
  def __init__(self, memoryLimit: int):
    self.size = memoryLimit
    self.packet2info: dict[int, tuple[int, int, int]] = {}
    self.destination2timestamps: dict[int, list[int]] = {}
    self.queue: deque = deque()

  def addPacket(self, source: int, destination: int, timestamp: int) -> bool:
    packet = self._encode_packet(source, destination, timestamp)

    if packet in self.packet2info:
      return False

    if len(self.packet2info) >= self.size:
      self.forwardPacket()

    self.packet2info[packet] = (source, destination, timestamp)
    if destination not in self.destination2timestamps:
      self.destination2timestamps[destination] = []
    self.destination2timestamps[destination].append(timestamp)
    self.queue.append(packet)

    return True

  def forwardPacket(self) -> List[int]:
    if not self.packet2info:
      return []

    packet = self.queue.popleft()
    source, destination, timestamp = self.packet2info.pop(packet)
    self.destination2timestamps[destination].pop(0)

    return [source, destination, timestamp]

  def getCount(self, destination: int, startTime: int, endTime: int) -> int:
    timestamps = self.destination2timestamps.get(destination, [])
    if not timestamps:
      return 0

    left = bisect.bisect_left(timestamps, startTime)
    right = bisect.bisect_right(timestamps, endTime)
    return right - left

  def _encode_packet(self, source: int, destination: int, timestamp: int) -> int:
    return (source << 40) | (destination << 20) | timestamp